In [1]:
import subprocess
import sys
from pathlib import Path

BASE_DIR = Path('/home/wencm/RecetONA')
REQ_PATH = BASE_DIR / 'requirements.txt'
LIB_DIR = BASE_DIR / 'lib'
LIB_DIR.mkdir(exist_ok=True)

subprocess.check_call([
    sys.executable,
    '-m',
    'pip',
    'install',
    '--upgrade',
    '--target',
    str(LIB_DIR),
    '-r',
    str(REQ_PATH),
])

print(f'Requirements instalados en: {LIB_DIR}')


  Using cached numpy-2.4.2-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (6.6 kB)
  Using cached pandas-3.0.0-cp312-cp312-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl.metadata (79 kB)
  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached openai-2.20.0-py3-none-any.whl.metadata (29 kB)
  Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl.metadata (8.4 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached anyio-4.12.1-py3-none-any.whl.metadata (4.3 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached jiter-0.13.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.2 kB)
  Using cached pydantic-2.12.5-py3-none-any.whl.metadata (90 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached tqdm-4.67.3-py3-none-any.whl.metadata (57 kB)
  Using cached typing_ext

# Mercadona RAG Notebook

Este notebook hace RAG sobre `mercadona_data.xlsx`, recupera filas relevantes y llama a OpenAI para responder usando solo ese contexto.

## Requisitos
- Tener `OPENAI_API_KEY` en variables de entorno.
- Tener `openai`, `pandas`, `numpy`, `openpyxl` instalados en `/home/wencm/RecetONA/lib`.


In [2]:
import os
import sys
from pathlib import Path

LIB_DIR = Path('/home/wencm/RecetONA/lib')
if str(LIB_DIR) not in sys.path:
    sys.path.insert(0, str(LIB_DIR))

import numpy as np
import pandas as pd
from openai import OpenAI

BASE_DIR = Path('/home/wencm/RecetONA')
EXCEL_PATH = BASE_DIR / 'mercadona_data.xlsx'
CACHE_DIR = BASE_DIR / 'rag_cache'
CACHE_DIR.mkdir(exist_ok=True)
ENV_PATH = BASE_DIR / '.env'

CHUNKS_CSV = CACHE_DIR / 'chunks.csv'
EMB_NPY = CACHE_DIR / 'embeddings.npy'

EMBED_MODEL = 'text-embedding-3-small'
CHAT_MODEL = 'gpt-4.1-mini'

def load_env_file(path: Path) -> bool:
    if not path.exists():
        return False

    loaded_any = False
    for raw_line in path.read_text(encoding='utf-8').splitlines():
        line = raw_line.strip()
        if not line or line.startswith('#') or '=' not in line:
            continue

        key, value = line.split('=', 1)
        key = key.strip()
        value = value.strip()

        if (value.startswith('"') and value.endswith('"')) or (value.startswith("'") and value.endswith("'")):
            value = value[1:-1]

        if key and key not in os.environ:
            os.environ[key] = value
            loaded_any = True

    return loaded_any

if not os.getenv('OPENAI_API_KEY'):
    loaded = load_env_file(ENV_PATH)
    if loaded:
        print(f'Variables cargadas desde: {ENV_PATH}')

print(f'Excel: {EXCEL_PATH}')
print(f'Cache: {CACHE_DIR}')
print(f'Python: {sys.executable}')
print('OPENAI_API_KEY presente:', bool(os.getenv('OPENAI_API_KEY')))



Excel: /home/wencm/RecetONA/mercadona_data.xlsx
Cache: /home/wencm/RecetONA/rag_cache
Python: /home/wencm/WhaRAGBot/.venv/bin/python
OPENAI_API_KEY presente: True


In [3]:
def _clean(v):
    if pd.isna(v):
        return ''
    s = str(v).strip()
    return '' if s.lower() == 'nan' else s

def _numeric(v):
    if pd.isna(v):
        return ''
    try:
        f = float(v)
    except Exception:
        return _clean(v)
    return f'{f:g}'

def build_row_text(row):
    parts = []
    parts.append(f"Producto: {_clean(row.get('product_name'))}")
    parts.append(f"ID producto: {_numeric(row.get('product_id'))}")
    parts.append(f"Categoria: {_clean(row.get('category'))} > {_clean(row.get('subcategory'))} > {_clean(row.get('subsubcategory'))}")
    parts.append(f"Formato: {_clean(row.get('packaging'))}, {_numeric(row.get('unit_size'))} {_clean(row.get('size_format'))}")
    parts.append(f"Precio unidad: {_numeric(row.get('price_unit'))} EUR")
    parts.append(f"Precio por volumen/peso: {_numeric(row.get('price_bulk'))} EUR")
    parts.append(f"Ingredientes: {_clean(row.get('Ingredientes'))}")
    parts.append(f"Alergenos: {_clean(row.get('Alérgenos'))}")
    parts.append(
        'Nutricion por 100g/ml: '
        f"kJ={_numeric(row.get('nutrition_kj_100'))}; "
        f"kcal={_numeric(row.get('nutrition_kcal_100'))}; "
        f"grasas={_numeric(row.get('nutrition_fat_g_100'))} g; "
        f"saturadas={_numeric(row.get('nutrition_saturates_g_100'))} g; "
        f"hidratos={_numeric(row.get('nutrition_carbs_g_100'))} g; "
        f"azucares={_numeric(row.get('nutrition_sugars_g_100'))} g; "
        f"proteinas={_numeric(row.get('nutrition_protein_g_100'))} g; "
        f"sal={_numeric(row.get('nutrition_salt_g_100'))} g"
    )
    parts.append(f"Imagen principal: {_clean(row.get('thumbnail_url'))}")
    parts.append(f"Imagenes: {_clean(row.get('photo_urls'))}")
    return '\n'.join(parts)

df = pd.read_excel(EXCEL_PATH)
print('Filas:', len(df))

chunks = pd.DataFrame({
    'row_idx': np.arange(len(df), dtype=int),
    'product_id': df.get('product_id'),
    'product_name': df.get('product_name'),
    'category': df.get('category'),
    'subcategory': df.get('subcategory'),
    'subsubcategory': df.get('subsubcategory'),
    'packaging': df.get('packaging'),
    'unit_size': df.get('unit_size'),
    'size_format': df.get('size_format'),
    'price_unit': df.get('price_unit'),
    'price_bulk': df.get('price_bulk'),
    'ingredientes': df.get('Ingredientes'),
    'alergenos': df.get('Alérgenos'),
    'nutrition_ocr_text': df.get('nutrition_ocr_text'),
    'text': [build_row_text(r) for _, r in df.iterrows()],
})

chunks['lexical_text'] = (
    chunks['product_name'].fillna('').astype(str) + ' ' +
    chunks['category'].fillna('').astype(str) + ' ' +
    chunks['subcategory'].fillna('').astype(str) + ' ' +
    chunks['subsubcategory'].fillna('').astype(str) + ' ' +
    chunks['ingredientes'].fillna('').astype(str)
)

# Ingredient-focused name/index fields (high precision)
chunks['ingredient_search_text'] = (
    chunks['product_name'].fillna('').astype(str) + ' ' +
    chunks['category'].fillna('').astype(str) + ' ' +
    chunks['subcategory'].fillna('').astype(str) + ' ' +
    chunks['subsubcategory'].fillna('').astype(str)
)

# Ingredient-focused description fields (high recall)
chunks['ingredient_desc_text'] = (
    chunks['ingredientes'].fillna('').astype(str) + ' ' +
    chunks['alergenos'].fillna('').astype(str) + ' ' +
    chunks['nutrition_ocr_text'].fillna('').astype(str).str.slice(0, 2500)
)

chunks.to_csv(CHUNKS_CSV, index=False)
print('Chunks guardados en:', CHUNKS_CSV)
chunks.head(2)


Filas: 4553
Chunks guardados en: /home/wencm/RecetONA/rag_cache/chunks.csv


,row_idx,product_id,product_name,category,subcategory,subsubcategory,packaging,unit_size,size_format,price_unit,price_bulk,ingredientes,alergenos,nutrition_ocr_text,text,lexical_text,ingredient_search_text,ingredient_desc_text
0,0,4241.0,"Aceite de oliva 0,4º Hacendado","Aceite, especias y salsas","Aceite, vinagre y sal",Aceite de oliva,Garrafa,5.0,l,19.75,3.95,Ingredientes: Aceite de oliva refinado y Aceit...,x99.,Aceitede Oliva | Que contiene exclusivamente |...,"Producto: Aceite de oliva 0,4º Hacendado\nID p...","Aceite de oliva 0,4º Hacendado Aceite, especia...","Aceite de oliva 0,4º Hacendado Aceite, especia...",Ingredientes: Aceite de oliva refinado y Aceit...
1,1,4240.0,"Aceite de oliva 0,4º Hacendado","Aceite, especias y salsas","Aceite, vinagre y sal",Aceite de oliva,Botella,1.0,l,4.10,4.10,Ingredientes: Aceite de oliva refinado y Aceit...,x99.,INGREDIENTES | CONSERVACION | Aceite de oliva ...,"Producto: Aceite de oliva 0,4º Hacendado\nID p...","Aceite de oliva 0,4º Hacendado Aceite, especia...","Aceite de oliva 0,4º Hacendado Aceite, especia...",Ingredientes: Aceite de oliva refinado y Aceit...


In [4]:
if not os.getenv('OPENAI_API_KEY'):
    raise RuntimeError(
        'Falta OPENAI_API_KEY. Define la variable en tu entorno o crea '
        f"{ENV_PATH}"
        ' con una linea: OPENAI_API_KEY=tu_clave'
    )

client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

def _l2_normalize(arr):
    norms = np.linalg.norm(arr, axis=1, keepdims=True) + 1e-12
    return arr / norms

def embed_texts(texts, model=EMBED_MODEL, batch_size=64):
    vectors = []
    for start in range(0, len(texts), batch_size):
        batch = texts[start:start + batch_size]
        resp = client.embeddings.create(model=model, input=batch)
        batch_vecs = [d.embedding for d in resp.data]
        vectors.extend(batch_vecs)
        print(f'Embeddings: {min(start + batch_size, len(texts))}/{len(texts)}')
    arr = np.array(vectors, dtype=np.float32)
    return _l2_normalize(arr)

def ensure_embeddings(chunks_df, force_rebuild=False):
    if EMB_NPY.exists() and not force_rebuild:
        emb = np.load(EMB_NPY)
        if emb.shape[0] == len(chunks_df):
            print('Embeddings cargados desde cache:', EMB_NPY)
            return emb
        print('Cache invalida por numero de filas. Se regenera.')

    emb = embed_texts(chunks_df['text'].tolist())
    np.save(EMB_NPY, emb)
    print('Embeddings guardados en:', EMB_NPY)
    return emb


In [5]:
embeddings = ensure_embeddings(chunks)
embeddings.shape


Embeddings cargados desde cache: /home/wencm/RecetONA/rag_cache/embeddings.npy


(4553, 1536)

In [6]:
import json
import math
import re
import unicodedata
from collections import Counter, defaultdict

SYSTEM_PROMPT = (
    'Eres un asistente experto en el catalogo de Mercadona. '
    'Responde SOLO con la informacion del contexto recuperado. '
    'Si no hay evidencia suficiente, di claramente que no aparece en el dataset. '
    'Si la pregunta es una receta, construye una lista de compra realista para el numero de personas '
    'usando productos presentes en el contexto, y separa siempre: '
    '1) coste de compra real (envases completos), '
    '2) escandallo real del plato (coste consumido). '
    'Si falta precio de algun ingrediente, indicalo explicitamente.'
)

RECIPE_TRIGGERS = (
    'receta', 'rehogado', 'rehogar', 'guiso', 'cocinar', 'ingredientes',
    'personas', 'menu', 'plato', 'preparar'
)

STOPWORDS = {
    'de', 'la', 'el', 'los', 'las', 'un', 'una', 'unos', 'unas', 'para', 'por', 'con', 'sin',
    'que', 'del', 'al', 'en', 'y', 'o', 'a', 'se', 'su', 'sus', 'como', 'dime', 'quiero',
    'hacer', 'preparar', 'receta', 'personas', 'plato', 'menu'
}

# Cantidades base estimadas para 4 personas (unidad compatible con dataset: kg/l/ud)
RECIPE_REQUIREMENTS_BASE_4P = {
    'lentejas': (0.40, 'kg'),
    'arroz': (0.32, 'kg'),
    'marisco': (0.45, 'kg'),
    'caldo de marisco': (1.00, 'l'),
    'caldo': (1.00, 'l'),
    'tomate': (0.20, 'kg'),
    'cebolla': (0.15, 'kg'),
    'ajo': (0.02, 'kg'),
    'pimiento': (0.15, 'kg'),
    'zanahoria': (0.15, 'kg'),
    'aceite de oliva': (0.06, 'l'),
    'huevo': (4.00, 'ud'),
    'huevos': (4.00, 'ud'),
    'sal': (0.01, 'kg'),
    'pimenton': (0.005, 'kg'),
    'laurel': (0.002, 'kg'),
}

def normalize_text(text):
    if text is None:
        return ''
    text = str(text).lower()
    text = unicodedata.normalize('NFD', text)
    return ''.join(ch for ch in text if unicodedata.category(ch) != 'Mn')

def _token_variants(token):
    variants = {token}
    if len(token) > 4 and token.endswith('es'):
        variants.add(token[:-2])
    if len(token) > 3 and token.endswith('s'):
        variants.add(token[:-1])
    return variants

def tokenize(text):
    raw = re.findall(r'[a-z0-9]+', normalize_text(text))
    out = []
    for tok in raw:
        out.extend(_token_variants(tok))
    return out

def _safe_float(v):
    try:
        if v is None:
            return None
        if isinstance(v, str) and not v.strip():
            return None
        return float(v)
    except Exception:
        return None

def _fmt_money(v):
    if v is None or (isinstance(v, float) and math.isnan(v)):
        return '-'
    return f'{v:.2f}'

def _fmt_num(v, digits=3):
    if v is None or (isinstance(v, float) and math.isnan(v)):
        return '-'
    return f'{v:.{digits}f}'

def _to_dim_base(value, unit):
    value = _safe_float(value)
    unit_n = normalize_text(unit)
    if value is None:
        return None, None

    if unit_n == 'kg':
        return 'mass_kg', value
    if unit_n == 'g':
        return 'mass_kg', value / 1000.0
    if unit_n == 'l':
        return 'vol_l', value
    if unit_n == 'ml':
        return 'vol_l', value / 1000.0
    if unit_n in ('ud', 'u', 'unidad', 'unidades'):
        return 'unit_ud', value
    return None, None

def parse_servings(query, default=4):
    qn = normalize_text(query)
    m = re.search(r'(\d+)\s*personas?', qn)
    if m:
        try:
            n = int(m.group(1))
            if n > 0:
                return n
        except Exception:
            pass
    return default

def is_recipe_query(query):
    qn = normalize_text(query)
    return any(trigger in qn for trigger in RECIPE_TRIGGERS)

def _remove_redundant_ingredients(items):
    normalized = [(item, normalize_text(item)) for item in items]
    keep = []

    for i, (item, norm_item) in enumerate(normalized):
        redundant = False
        for j, (_, norm_other) in enumerate(normalized):
            if i == j:
                continue
            if norm_item in norm_other and len(norm_item.split()) < len(norm_other.split()):
                redundant = True
                break
        if not redundant:
            keep.append(item)

    return keep

def _extract_json_list(text):
    text = text.strip()
    if not text:
        return []

    # 1) Try raw JSON first
    try:
        obj = json.loads(text)
        if isinstance(obj, list):
            return obj
        if isinstance(obj, dict) and isinstance(obj.get('ingredients'), list):
            return obj['ingredients']
    except Exception:
        pass

    # 2) Try to capture first JSON list in markdown or prose
    m = re.search(r'\[[\s\S]*\]', text)
    if m:
        try:
            obj = json.loads(m.group(0))
            if isinstance(obj, list):
                return obj
        except Exception:
            pass

    return []

def _normalize_ingredient_alias(s):
    s = normalize_text(s)
    if not s:
        return s

    alias_rules = [
        (r'\bpolvos?\s+(de|para)\s+hornear\b', 'gasificante reposteria'),
        (r'\blevadura\s+quimica\b', 'gasificante reposteria'),
        (r'\bimpulsor(?:\s+gasificante)?\b', 'gasificante reposteria'),
        (r'\bextracto\s+de\s+vainilla\b', 'aroma de vainilla'),
        (r'\bcrema\s+de\s+leche\b', 'nata'),
        (r'\bcrema\s+para\s+batir\b', 'nata'),
        (r'\bnata\s+liquida\b', 'nata'),
    ]

    for pattern, replacement in alias_rules:
        if re.search(pattern, s):
            return replacement

    return s


def _clean_ingredient_name(item):
    s = normalize_text(item)
    s = re.sub(r'\([^)]*\)', ' ', s)
    s = re.sub(r'\baprox(?:imadamente)?\b', ' ', s)

    # Remove leading quantity/unit chunks: "400 g de", "2 dientes de", etc.
    s = re.sub(
        r'^\s*\d+(?:[\.,]\d+)?\s*(kg|g|gr|gramos?|ml|l|litros?|unidad(?:es)?|ud|dientes?|cucharad(?:a|as)|taza(?:s)?)?\s*(de\s+)?',
        '',
        s,
    )
    s = re.sub(r'^\s*(unos?|unas?|un|una)\s+', '', s)

    # Remove residual quantities/units inside phrase.
    s = re.sub(r'\b\d+(?:[\.,]\d+)?\b', ' ', s)
    s = re.sub(r'\b(kg|g|gr|gramos?|ml|l|litros?|unidad(?:es)?|ud|dientes?|cucharad(?:a|as)|taza(?:s)?)\b', ' ', s)

    s = re.sub(r'\s+', ' ', s).strip()
    if s.startswith('de '):
        s = s[3:].strip()

    return _normalize_ingredient_alias(s)

def infer_recipe_ingredients_llm(query, max_items=12):
    prompt = (
        'Extrae ingredientes para la receta solicitada. '
        'Devuelve SOLO un JSON array de strings, sin texto adicional. '
        'No incluyas cantidades ni unidades. '
        'Incluye ingredientes base y condimentos habituales. '
        'No incluyas bebidas, refrescos ni acompanamientos opcionales. '
        f'Consulta: {query}'
    )

    try:
        resp = client.responses.create(
            model=CHAT_MODEL,
            input=prompt,
        )
        raw = (resp.output_text or '').strip()
        arr = _extract_json_list(raw)

        out = []
        seen = set()
        for x in arr:
            if not isinstance(x, str):
                continue
            item = _clean_ingredient_name(x)
            if len(item) < 2:
                continue
            key = normalize_text(item)
            if not key or key in seen:
                continue
            seen.add(key)
            out.append(item)
            if len(out) >= max_items:
                break

        return out
    except Exception as exc:
        print(f'WARN infer_recipe_ingredients_llm fallo: {exc}')
        return []

def infer_recipe_ingredients(query):
    if not is_recipe_query(query):
        return []

    inferred = infer_recipe_ingredients_llm(query)
    if inferred:
        return _remove_redundant_ingredients(inferred)

    # Fallback sin reglas de plato concreto
    tokens = [
        t for t in tokenize(query)
        if len(t) > 2 and t not in STOPWORDS and not t.isdigit()
    ]
    return _remove_redundant_ingredients(tokens[:8])

def _ingredient_requirement_key(ingredient):
    ing_n = normalize_text(ingredient)
    keys = sorted(RECIPE_REQUIREMENTS_BASE_4P.keys(), key=len, reverse=True)
    for key in keys:
        if key in ing_n:
            return key
    return None

def estimate_required_quantity(ingredient, servings=4):
    key = _ingredient_requirement_key(ingredient)
    if key is None:
        return 1.0 * (servings / 4.0), 'ud', 'fallback'

    base_qty, base_unit = RECIPE_REQUIREMENTS_BASE_4P[key]
    factor = servings / 4.0
    return base_qty * factor, base_unit, key

def expand_subqueries(query, recipe_mode='auto'):
    subqueries = [query]
    inferred_ingredients = []

    use_recipe = (recipe_mode is True) or (recipe_mode == 'auto' and is_recipe_query(query))
    if use_recipe:
        inferred_ingredients = infer_recipe_ingredients(query)
        for ingredient in inferred_ingredients:
            subqueries.append(f'{ingredient} mercadona precio')

    return subqueries, inferred_ingredients

# General index (RAG context)
DOC_TOKENS = [tokenize(text) for text in chunks['lexical_text'].fillna('')]
POSTINGS = defaultdict(list)
DOC_FREQ = defaultdict(int)
N_DOCS = len(DOC_TOKENS)

for doc_idx, toks in enumerate(DOC_TOKENS):
    counts = Counter(toks)
    for tok, tf in counts.items():
        POSTINGS[tok].append((doc_idx, tf))
        DOC_FREQ[tok] += 1

# Ingredient-focused indexes
# 1) Name/category index (precision)
ING_NAME_DOC_TOKENS = [tokenize(text) for text in chunks['ingredient_search_text'].fillna('')]
ING_NAME_TOKEN_SETS = [set(toks) for toks in ING_NAME_DOC_TOKENS]
ING_NAME_POSTINGS = defaultdict(list)
ING_NAME_DOC_FREQ = defaultdict(int)

for doc_idx, toks in enumerate(ING_NAME_DOC_TOKENS):
    counts = Counter(toks)
    for tok, tf in counts.items():
        ING_NAME_POSTINGS[tok].append((doc_idx, tf))
        ING_NAME_DOC_FREQ[tok] += 1

# 2) Description/ingredients index (recall)
ING_DESC_DOC_TOKENS = [tokenize(text) for text in chunks['ingredient_desc_text'].fillna('')]
ING_DESC_TOKEN_SETS = [set(toks) for toks in ING_DESC_DOC_TOKENS]
ING_DESC_POSTINGS = defaultdict(list)
ING_DESC_DOC_FREQ = defaultdict(int)

for doc_idx, toks in enumerate(ING_DESC_DOC_TOKENS):
    counts = Counter(toks)
    for tok, tf in counts.items():
        ING_DESC_POSTINGS[tok].append((doc_idx, tf))
        ING_DESC_DOC_FREQ[tok] += 1

def _tfidf_scores_index(q_tokens, postings, doc_freq):
    if not q_tokens:
        return np.zeros(N_DOCS, dtype=np.float32)

    q_counts = Counter(q_tokens)
    scores = np.zeros(N_DOCS, dtype=np.float32)

    for tok, qtf in q_counts.items():
        rows = postings.get(tok)
        if not rows:
            continue

        idf = math.log((N_DOCS + 1) / (doc_freq[tok] + 1)) + 1.0
        q_weight = 1.0 + math.log(qtf)
        for doc_i, tf in rows:
            scores[doc_i] += q_weight * (1.0 + math.log(tf)) * idf

    max_score = float(scores.max())
    if max_score > 0:
        scores /= max_score
    return scores

def lexical_scores(query):
    q_tokens = [t for t in tokenize(query) if len(t) > 2 and t not in STOPWORDS]
    return _tfidf_scores_index(q_tokens, POSTINGS, DOC_FREQ)

def lexical_scores_ingredient(query):
    q_tokens = [t for t in tokenize(query) if len(t) > 2 and t not in STOPWORDS]
    name_scores = _tfidf_scores_index(q_tokens, ING_NAME_POSTINGS, ING_NAME_DOC_FREQ)
    desc_scores = _tfidf_scores_index(q_tokens, ING_DESC_POSTINGS, ING_DESC_DOC_FREQ)

    # Name/category drives precision; ingredients/description improves recall for indirect product names.
    return 0.80 * name_scores + 0.20 * desc_scores

def semantic_scores_batch(queries):
    resp = client.embeddings.create(model=EMBED_MODEL, input=queries)
    all_scores = []
    for item in resp.data:
        q_vec = np.array(item.embedding, dtype=np.float32)
        q_vec = q_vec / (np.linalg.norm(q_vec) + 1e-12)
        sem = embeddings @ q_vec
        all_scores.append((sem + 1.0) / 2.0)
    return all_scores

def _ingredient_token_masks(ingredient):
    tokens = [t for t in tokenize(ingredient) if t not in STOPWORDS and len(t) > 2]
    if not tokens:
        zeros = np.zeros(N_DOCS, dtype=bool)
        return zeros, zeros

    any_mask = np.zeros(N_DOCS, dtype=bool)
    all_mask = np.zeros(N_DOCS, dtype=bool)

    for doc_i, tokset in enumerate(ING_NAME_TOKEN_SETS):
        present = [tok in tokset for tok in tokens]
        if any(present):
            any_mask[doc_i] = True
        if all(present):
            all_mask[doc_i] = True

    return any_mask, all_mask

def retrieve_hybrid(query, top_k=20, alpha=0.65, recipe_mode='auto'):
    subqueries, inferred_ingredients = expand_subqueries(query, recipe_mode=recipe_mode)
    sem_scores_list = semantic_scores_batch(subqueries)

    hybrid_best = np.zeros(N_DOCS, dtype=np.float32)
    semantic_best = np.zeros(N_DOCS, dtype=np.float32)
    lexical_best = np.zeros(N_DOCS, dtype=np.float32)

    for subquery, sem_scores in zip(subqueries, sem_scores_list):
        lex_scores = lexical_scores(subquery)
        hybrid_scores = alpha * sem_scores + (1.0 - alpha) * lex_scores

        hybrid_best = np.maximum(hybrid_best, hybrid_scores)
        semantic_best = np.maximum(semantic_best, sem_scores)
        lexical_best = np.maximum(lexical_best, lex_scores)

    top_idx = np.argsort(-hybrid_best)[:top_k]

    hits = chunks.iloc[top_idx].copy()
    hits['score'] = hybrid_best[top_idx]
    hits['score_semantic'] = semantic_best[top_idx]
    hits['score_lexical'] = lexical_best[top_idx]

    return hits.sort_values('score', ascending=False).reset_index(drop=True), subqueries, inferred_ingredients

def retrieve(query, top_k=8, mode='hybrid', alpha=0.65, recipe_mode='auto'):
    mode = mode.lower()

    if mode == 'semantic':
        sem_scores = semantic_scores_batch([query])[0]
        top_idx = np.argsort(-sem_scores)[:top_k]
        hits = chunks.iloc[top_idx].copy()
        hits['score'] = sem_scores[top_idx]
        return hits.sort_values('score', ascending=False).reset_index(drop=True), [query], []

    if mode == 'hybrid':
        return retrieve_hybrid(query, top_k=top_k, alpha=alpha, recipe_mode=recipe_mode)

    raise ValueError("mode debe ser 'semantic' o 'hybrid'")

def retrieve_products_for_ingredient(ingredient, top_n=10, alpha=0.35, recipe_query=None):
    ingredient_query = normalize_text(ingredient).strip()
    context_query = normalize_text(recipe_query).strip() if recipe_query else ''

    cols = [
        'row_idx', 'product_id', 'product_name', 'category', 'subcategory',
        'packaging', 'unit_size', 'size_format', 'price_unit', 'price_bulk'
    ]

    if not ingredient_query:
        empty = pd.DataFrame(columns=cols)
        empty['ingredient'] = []
        empty['score_ingredient'] = []
        return empty

    # Ingredient retrieval should be driven by ingredient evidence, not by full recipe wording.
    sem_ingredient = semantic_scores_batch([f'{ingredient_query} mercadona ingrediente'])[0]
    lex_scores = lexical_scores_ingredient(ingredient_query)

    # Safety: if there is no lexical evidence at all, return empty instead of noisy semantic matches.
    if float(lex_scores.max()) <= 0.0:
        empty = pd.DataFrame(columns=cols)
        empty['ingredient'] = []
        empty['score_ingredient'] = []
        return empty

    # Optional weak context signal to disambiguate when several ingredient candidates are valid.
    if context_query:
        sem_context = semantic_scores_batch([context_query])[0]
    else:
        sem_context = np.zeros(N_DOCS, dtype=np.float32)

    base_scores = alpha * sem_ingredient + (1.0 - alpha) * lex_scores
    hybrid_scores = 0.90 * base_scores + 0.10 * sem_context

    any_mask, all_mask = _ingredient_token_masks(ingredient_query)
    if any_mask.any():
        hybrid_scores = np.where(any_mask, hybrid_scores + 0.25, hybrid_scores * 0.10)
    if all_mask.any():
        hybrid_scores = np.where(all_mask, hybrid_scores + 0.15, hybrid_scores)

    candidate_n = min(N_DOCS, max(top_n * 8, top_n))
    top_idx = np.argsort(-hybrid_scores)[:candidate_n]

    out_cols = cols + ['ingredientes']
    out = chunks.iloc[top_idx][out_cols].copy()
    out['ingredient'] = ingredient
    out['score_ingredient'] = hybrid_scores[top_idx]

    out['category_text'] = (
        out['category'].fillna('').astype(str).map(normalize_text)
        + ' '
        + out['subcategory'].fillna('').astype(str).map(normalize_text)
    )

    non_food_regex = re.compile(
        r'\b(maquill|cuidado|perfume|colonia|higiene|limpieza|hogar|bebe|mascota|'
        r'parafarm|deterg|desodor|capilar|depil|afeitad|insectic|lavavaj|suavizante|'
        r'papel|celulosa|escoba|fregona|ambientador)\b'
    )

    non_food_mask = out['category_text'].apply(lambda txt: bool(non_food_regex.search(txt)))
    if (~non_food_mask).any():
        out = out[~non_food_mask].copy()

    ing_tokens = {
        t for t in tokenize(ingredient_query)
        if len(t) > 2 and t not in STOPWORDS
    }

    descriptor_tokens = STOPWORDS | {
        'hacendado', 'mercadona', 'valor', 'marca', 'extra', 'ultra', 'ultracongelado',
        'congelado', 'fresco', 'fresca', 'fino', 'fina', 'grueso', 'gruesa', 'mesa',
        'natural', 'clasico', 'clasica', 'entero', 'entera', 'semidesnatada', 'desnatada',
        'kg', 'g', 'gr', 'ml', 'l', 'litro', 'litros', 'ud', 'u', 'pack', 'paquete',
        'bote', 'botella', 'lata', 'tarro', 'bolsa'
    }

    def _candidate_penalty(row):
        if not ing_tokens:
            return 0.0

        product_name = normalize_text(row.get('product_name'))
        name_tokens = {
            t for t in tokenize(product_name)
            if len(t) > 2 and t not in STOPWORDS
        }

        ingredientes_text = normalize_text(row.get('ingredientes'))
        ingredient_list_tokens = {
            t for t in tokenize(ingredientes_text)
            if len(t) > 2 and t not in STOPWORDS
        }

        category_text = normalize_text(row.get('category')) + ' ' + normalize_text(row.get('subcategory'))

        penalty = 0.0

        coverage = len(ing_tokens & name_tokens) / max(1, len(ing_tokens))
        if coverage < 1.0:
            penalty += (1.0 - coverage) * 0.45

        # Penalize strongly processed/composite naming patterns for base ingredients.
        if ',' in product_name:
            penalty += 0.12
        if ' y ' in f' {product_name} ':
            penalty += 0.10
        if re.search(r'\b(sabor|rellen|mix|fusion|cocktail|preparad|listo)\b', product_name):
            penalty += 0.20

        # For short ingredient names, aggressively penalize unrelated qualifiers.
        foreign_name_tokens = [
            t for t in name_tokens
            if t not in ing_tokens and t not in descriptor_tokens
        ]
        if len(ing_tokens) <= 1:
            penalty += min(0.70, 0.32 * len(foreign_name_tokens))
        else:
            penalty += min(0.20, 0.05 * len(foreign_name_tokens))

        if ingredient_list_tokens:
            foreign_ingredient_tokens = [
                t for t in ingredient_list_tokens
                if t not in ing_tokens and t not in descriptor_tokens
            ]
            if len(ing_tokens) <= 1:
                penalty += min(0.45, 0.015 * len(foreign_ingredient_tokens))
            else:
                penalty += min(0.22, 0.010 * len(foreign_ingredient_tokens))

        # Generic ingredient disambiguation rules (not recipe-specific):
        # - plain egg should avoid quail/claras/cooked variants
        if ('huevo' in ing_tokens or 'huevos' in ing_tokens) and re.search(r'\b(codorniz|claras?|cocid[oa]s?)\b', product_name):
            penalty += 0.40

        # - dairy cream intent should avoid vegetable/soup creams
        dairy_cream_intent = (
            ('crema' in ing_tokens and 'leche' in ing_tokens)
            or ('nata' in ing_tokens)
        )
        if dairy_cream_intent:
            if re.search(r'\b(gazpacho|sopa|cremas?)\b', category_text):
                penalty += 1.20
            if re.search(r'\b(calabaza|calabacin|verduras?|pollo|setas?)\b', product_name):
                penalty += 1.20

        return float(penalty)

    out['candidate_penalty'] = out.apply(_candidate_penalty, axis=1)
    out['score_ingredient'] = out['score_ingredient'] - out['candidate_penalty']

    # Deduplicate same product across duplicated category branches.
    out = out.sort_values('score_ingredient', ascending=False).drop_duplicates(subset=['product_id'], keep='first')

    final_cols = cols + ['ingredient', 'score_ingredient']
    return out[final_cols].head(top_n).reset_index(drop=True)

def tool_get_products_for_ingredients(ingredients, per_ingredient=10, alpha=0.35, recipe_query=None):
    """
    Tool-like helper: for each ingredient, returns a ranked product shortlist.
    """
    catalog = {}
    for ingredient in ingredients:
        hits = retrieve_products_for_ingredient(ingredient, top_n=per_ingredient, alpha=alpha, recipe_query=recipe_query)
        catalog[ingredient] = hits
    return catalog

def _choose_best_candidate(df_hits, required_qty, required_unit, requirement_source='fallback'):
    if df_hits is None or df_hits.empty:
        return None

    req_dim, req_base = _to_dim_base(required_qty, required_unit)
    source = normalize_text(requirement_source)
    consider_qty = (source != 'fallback') and (req_dim is not None) and (req_base is not None) and (req_base > 0)

    ranked = []

    for _, row in df_hits.iterrows():
        price = _safe_float(row.get('price_unit'))
        if price is None:
            continue

        score = _safe_float(row.get('score_ingredient'))
        if score is None:
            score = 0.0

        unit_size = _safe_float(row.get('unit_size'))
        size_fmt = row.get('size_format')
        pack_dim, pack_base = _to_dim_base(unit_size, size_fmt)

        comparable = consider_qty and (pack_dim == req_dim) and (pack_base is not None) and (pack_base > 0)
        if comparable:
            units_to_buy = max(1, math.ceil(req_base / pack_base))
            purchase_cost = units_to_buy * price
        else:
            units_to_buy = 1
            purchase_cost = price

        ranked.append({
            'row': row,
            'score': float(score),
            'comparable': bool(comparable),
            'units_to_buy': int(units_to_buy),
            'purchase_cost': float(purchase_cost),
        })

    if not ranked:
        return df_hits.iloc[0]

    if consider_qty:
        comparable_rows = [x for x in ranked if x['comparable']]
        if comparable_rows:
            comparable_rows.sort(key=lambda x: (x['purchase_cost'], x['units_to_buy'], -x['score']))
            return comparable_rows[0]['row']

    # Fallback: keep relevance first, then prefer cheaper package.
    ranked.sort(key=lambda x: (-x['score'], x['purchase_cost']))
    return ranked[0]['row']

def _compute_buy_and_escandallo(chosen_row, required_qty, required_unit):
    if chosen_row is None:
        return {
            'units_to_buy': None,
            'purchase_cost_eur': None,
            'escandallo_cost_eur': None,
            'notes': 'Sin producto candidato',
        }

    price_unit = _safe_float(chosen_row.get('price_unit'))
    unit_size = _safe_float(chosen_row.get('unit_size'))
    size_format = chosen_row.get('size_format')

    req_dim, req_base = _to_dim_base(required_qty, required_unit)
    pack_dim, pack_base = _to_dim_base(unit_size, size_format)

    if price_unit is None:
        return {
            'units_to_buy': None,
            'purchase_cost_eur': None,
            'escandallo_cost_eur': None,
            'notes': 'Producto sin price_unit',
        }

    # Regla clave: compra siempre envase completo (redondeo al alza)
    if req_dim is not None and pack_dim == req_dim and pack_base is not None and pack_base > 0 and req_base is not None:
        units_to_buy = max(1, math.ceil(req_base / pack_base))
        purchase_cost = units_to_buy * price_unit
        escandallo_cost = (req_base / pack_base) * price_unit
        return {
            'units_to_buy': units_to_buy,
            'purchase_cost_eur': purchase_cost,
            'escandallo_cost_eur': escandallo_cost,
            'notes': 'OK unidades comparables',
        }

    # Fallback: no se puede convertir unidades -> compra minima 1 unidad.
    return {
        'units_to_buy': 1,
        'purchase_cost_eur': price_unit,
        'escandallo_cost_eur': None,
        'notes': 'Unidad no comparable, compra minima 1 unidad',
    }

def _dedupe_plan_rows_by_product(plan_df):
    if plan_df is None or plan_df.empty or 'product_id' not in plan_df.columns:
        return plan_df

    df = plan_df.copy()
    df['_row_order'] = np.arange(len(df), dtype=int)

    with_product = df[df['product_id'].notna()].copy()
    without_product = df[df['product_id'].isna()].copy()

    if with_product.empty:
        return df.drop(columns=['_row_order'], errors='ignore').reset_index(drop=True)

    with_product['_required_qty_num'] = with_product['required_qty'].apply(_safe_float)
    with_product['_required_qty_num'] = with_product['_required_qty_num'].fillna(-1.0)

    with_product = with_product.sort_values(
        by=['product_id', '_required_qty_num', '_row_order'],
        ascending=[True, False, True],
    ).drop_duplicates(subset=['product_id'], keep='first')

    collapsed = pd.concat([with_product, without_product], ignore_index=True, sort=False)
    collapsed = collapsed.sort_values('_row_order').drop(
        columns=['_row_order', '_required_qty_num'],
        errors='ignore',
    )

    return collapsed.reset_index(drop=True)


def build_recipe_cost_plan(ingredient_catalog, ingredients, query):
    servings = parse_servings(query, default=4)
    rows = []

    for ingredient in ingredients:
        required_qty, required_unit, req_source = estimate_required_quantity(ingredient, servings=servings)
        hits = ingredient_catalog.get(ingredient)
        chosen = _choose_best_candidate(hits, required_qty, required_unit, req_source)

        if chosen is None:
            rows.append({
                'ingredient': ingredient,
                'required_qty': required_qty,
                'required_unit': required_unit,
                'requirement_source': req_source,
                'product_id': None,
                'product_name': None,
                'unit_size': None,
                'size_format': None,
                'price_unit': None,
                'units_to_buy': None,
                'purchase_cost_eur': None,
                'escandallo_cost_eur': None,
                'notes': 'Sin candidatos',
            })
            continue

        cost_info = _compute_buy_and_escandallo(chosen, required_qty, required_unit)
        rows.append({
            'ingredient': ingredient,
            'required_qty': required_qty,
            'required_unit': required_unit,
            'requirement_source': req_source,
            'product_id': chosen.get('product_id'),
            'product_name': chosen.get('product_name'),
            'unit_size': _safe_float(chosen.get('unit_size')),
            'size_format': chosen.get('size_format'),
            'price_unit': _safe_float(chosen.get('price_unit')),
            'units_to_buy': cost_info['units_to_buy'],
            'purchase_cost_eur': cost_info['purchase_cost_eur'],
            'escandallo_cost_eur': cost_info['escandallo_cost_eur'],
            'notes': cost_info['notes'],
        })

    plan_df = pd.DataFrame(rows)
    plan_df = _dedupe_plan_rows_by_product(plan_df)

    total_purchase = _safe_float(plan_df['purchase_cost_eur'].dropna().sum()) if not plan_df.empty else 0.0
    total_escandallo = _safe_float(plan_df['escandallo_cost_eur'].dropna().sum()) if not plan_df.empty else 0.0

    missing_purchase = plan_df[plan_df['purchase_cost_eur'].isna()]['ingredient'].tolist() if not plan_df.empty else []
    missing_esc = plan_df[plan_df['escandallo_cost_eur'].isna()]['ingredient'].tolist() if not plan_df.empty else []

    summary = {
        'servings': servings,
        'total_purchase_eur': total_purchase,
        'total_escandallo_eur': total_escandallo,
        'missing_purchase_ingredients': missing_purchase,
        'missing_escandallo_ingredients': missing_esc,
    }

    return plan_df, summary

def format_ingredient_catalog_text(catalog, max_items=6):
    blocks = []
    for ingredient, df_hits in catalog.items():
        lines = [f'Ingrediente: {ingredient}']
        if df_hits.empty:
            lines.append('- Sin candidatos en el dataset')
        else:
            for _, row in df_hits.head(max_items).iterrows():
                lines.append(
                    '- '
                    f"product_id={row['product_id']}; "
                    f"nombre={row['product_name']}; "
                    f"categoria={row['category']}; "
                    f"precio_unit={row['price_unit']}; "
                    f"formato={row['unit_size']} {row['size_format']}; "
                    f"score={row['score_ingredient']:.4f}"
                )
        blocks.append('\n'.join(lines))
    return '\n\n'.join(blocks)

def format_cost_plan_text(plan_df, summary):
    if plan_df is None or plan_df.empty:
        return 'Sin plan de costes.'

    lines = []
    lines.append(f"Personas: {summary['servings']}")
    lines.append(f"TOTAL_COMPRA_EUR (envase completo): {_fmt_money(summary['total_purchase_eur'])}")
    lines.append(f"TOTAL_ESCANDALLO_EUR (consumo real): {_fmt_money(summary['total_escandallo_eur'])}")

    if summary['missing_purchase_ingredients']:
        lines.append('Faltan precios de compra para: ' + ', '.join(summary['missing_purchase_ingredients']))
    if summary['missing_escandallo_ingredients']:
        lines.append('No se pudo calcular escandallo para: ' + ', '.join(summary['missing_escandallo_ingredients']))

    lines.append('')
    lines.append('| ingrediente | product_id | producto | req_qty | pack_size | price_unit | units_to_buy | purchase_cost_eur | escandallo_cost_eur |')
    lines.append('|---|---:|---|---:|---|---:|---:|---:|---:|')

    for _, r in plan_df.iterrows():
        lines.append(
            f"| {r.get('ingredient')} | {r.get('product_id')} | {r.get('product_name')} | "
            f"{_fmt_num(r.get('required_qty'), 3)} {r.get('required_unit')} | "
            f"{_fmt_num(r.get('unit_size'), 3)} {r.get('size_format')} | "
            f"{_fmt_money(r.get('price_unit'))} | {r.get('units_to_buy')} | "
            f"{_fmt_money(r.get('purchase_cost_eur'))} | {_fmt_money(r.get('escandallo_cost_eur'))} |"
        )

    return '\n'.join(lines)


def _fmt_money_es(v):
    n = _safe_float(v)
    if n is None:
        return 'N/D'
    return f"{n:,.2f}".replace(',', 'X').replace('.', ',').replace('X', '.')

def _fmt_recipe_quantity_number(v, decimals=2):
    n = _safe_float(v)
    if n is None:
        return 'N/D'

    rounded = round(n)
    if abs(n - rounded) < 1e-9:
        return str(int(rounded))

    return f"{n:.{decimals}f}".rstrip('0').rstrip('.')


def _format_qty_for_recipe_line(ingredient, required_qty, required_unit):
    qty = _safe_float(required_qty)
    unit = normalize_text(required_unit)
    ing = normalize_text(ingredient)

    if qty is None:
        return 'cantidad N/D'

    condiments = ('sal', 'pimienta', 'azafran', 'pimenton', 'laurel')

    if unit == 'kg':
        grams = qty * 1000.0
        if any(c in ing for c in condiments):
            if grams <= 6:
                return '1 cucharadita'
            if grams <= 20:
                return '1 cucharada'
        if grams < 1000:
            return f"{int(round(grams))} g"
        return f"{_fmt_recipe_quantity_number(qty, 2)} kg"

    if unit == 'l':
        ml = qty * 1000.0
        if 'limon' in ing and ml <= 60:
            return '1 chorro'
        if 'aceite' in ing and ml <= 80:
            return '1 chorro'
        if ml < 1000:
            return f"{int(round(ml))} ml"
        return f"{_fmt_recipe_quantity_number(qty, 2)} L"

    if unit == 'ud':
        if 'limon' in ing:
            return '1 chorro'
        n = max(1, int(round(qty)))
        return f"{n} ud"

    unit_raw = str(required_unit).strip() if required_unit is not None else ''
    if unit_raw:
        return f"{_fmt_recipe_quantity_number(qty, 2)} {unit_raw}"
    return _fmt_recipe_quantity_number(qty, 2)

def build_block1_ingredients_mercadona(plan_df):
    if plan_df is None or plan_df.empty:
        return 'No se encontraron ingredientes/productos en Mercadona para esta receta.'

    lines = []
    seen_products = set()

    for _, r in plan_df.iterrows():
        ingredient = r.get('ingredient')
        product_name = r.get('product_name')
        req_qty = r.get('required_qty')
        req_unit = r.get('required_unit')
        product_id = r.get('product_id')
        purchase_cost = _safe_float(r.get('purchase_cost_eur'))
        units_to_buy = _safe_float(r.get('units_to_buy'))

        if product_name is None or str(product_name) == 'nan':
            lines.append(f"- {ingredient}: sin producto identificado en el dataset")
            continue

        if product_id is None or str(product_id) == 'nan':
            product_key = f"name:{normalize_text(product_name)}"
        else:
            product_key = f"id:{str(product_id).strip()}"

        if product_key in seen_products:
            continue

        seen_products.add(product_key)

        qty_text = _format_qty_for_recipe_line(ingredient, req_qty, req_unit)

        if purchase_cost is None:
            price_text = 'N/D'
        else:
            price_text = f"{_fmt_money_es(purchase_cost)}€"

        if units_to_buy is not None and units_to_buy > 1:
            units_text = f" (x{int(round(units_to_buy))} envases)"
        else:
            units_text = ''

        lines.append(f"- {qty_text} {product_name} ({price_text}){units_text}")

    return '\n'.join(lines)

def tool_get_total_purchase_price(cost_summary):
    return ''

def _build_recipe_prompt_context(plan_df, max_items=14):
    if plan_df is None or plan_df.empty:
        return 'No hay productos concretos disponibles.'

    lines = []
    for _, r in plan_df.head(max_items).iterrows():
        product_name = r.get('product_name')
        ingredient = r.get('ingredient')
        req_qty = _fmt_num(r.get('required_qty'), 3)
        req_unit = r.get('required_unit')
        if product_name is None or str(product_name) == 'nan':
            continue
        lines.append(f"- {ingredient}: {product_name} ({req_qty} {req_unit})")

    return '\n'.join(lines) if lines else 'No hay productos concretos disponibles.'

def build_block3_recipe_text(query, plan_df, model=CHAT_MODEL, max_chars=1000):
    context_lines = _build_recipe_prompt_context(plan_df)

    prompt = (
        f"Escribe una receta breve en español para esta solicitud: {query}.\n"
        f"Usa de referencia estos ingredientes/productos:\n{context_lines}\n\n"
        f"Reglas estrictas:\n"
        f"- Máximo {max_chars} caracteres.\n"
        f"- Texto corrido con pasos claros (sin tablas).\n"
        f"- No inventes precios.\n"
        f"- Devuelve solo el texto de la receta."
    )

    raw_resp = None
    text = ''
    try:
        raw_resp = client.responses.create(model=model, input=prompt)
        text = (raw_resp.output_text or '').strip()
    except Exception as exc:
        text = f"No se pudo generar el texto de receta automáticamente ({exc})."

    if len(text) > max_chars:
        text = text[: max_chars - 1].rstrip() + '…'

    return text, raw_resp

def compose_structured_answer(block1, block2, block3):
    parts = [
        'Ingredientes de Mercadona:',
        str(block1).strip(),
    ]

    block2_text = str(block2).strip() if block2 is not None else ''
    if block2_text:
        parts.append(block2_text)

    parts.extend([
        'Receta y pasos:',
        str(block3).strip(),
    ])

    return '\n\n'.join(parts)

def _catalog_preview(catalog, n=3):
    preview = {}
    for ing, df_hits in catalog.items():
        preview[ing] = [
            {
                'product_id': r['product_id'],
                'product_name': r['product_name'],
                'price_unit': r['price_unit'],
            }
            for _, r in df_hits.head(n).iterrows()
        ]
    return preview

def ask_agent(
    query,
    top_k=20,
    model=CHAT_MODEL,
    retrieval_mode='hybrid',
    alpha=0.65,
    recipe_mode='auto',
    use_ingredient_tool=True,
    candidates_per_ingredient=10,
):
    hits, subqueries, inferred_ingredients = retrieve(
        query,
        top_k=top_k,
        mode=retrieval_mode,
        alpha=alpha,
        recipe_mode=recipe_mode,
    )

    ingredient_catalog = {}
    ingredient_catalog_text = ''
    cost_plan_df = pd.DataFrame()
    cost_summary = {
        'servings': parse_servings(query, default=4),
        'total_purchase_eur': 0.0,
        'total_escandallo_eur': 0.0,
        'missing_purchase_ingredients': [],
        'missing_escandallo_ingredients': [],
    }
    cost_plan_text = 'Sin plan de costes.'

    if use_ingredient_tool and inferred_ingredients:
        ingredient_catalog = tool_get_products_for_ingredients(
            inferred_ingredients,
            per_ingredient=candidates_per_ingredient,
            alpha=0.35,
            recipe_query=query,
        )
        ingredient_catalog_text = format_ingredient_catalog_text(ingredient_catalog, max_items=6)

        cost_plan_df, cost_summary = build_recipe_cost_plan(
            ingredient_catalog=ingredient_catalog,
            ingredients=inferred_ingredients,
            query=query,
        )
        cost_plan_text = format_cost_plan_text(cost_plan_df, cost_summary)

    block_1 = build_block1_ingredients_mercadona(cost_plan_df)
    block_2 = tool_get_total_purchase_price(cost_summary)
    block_3, raw_resp_block3 = build_block3_recipe_text(query, cost_plan_df, model=model, max_chars=1000)
    structured_answer = compose_structured_answer(block_1, block_2, block_3)

    hit_cols = [
        'product_id', 'product_name', 'category', 'price_unit',
        'unit_size', 'size_format', 'score'
    ]
    for extra in ('score_semantic', 'score_lexical'):
        if extra in hits.columns:
            hit_cols.append(extra)

    return {
        'answer': structured_answer,
        'block_1': block_1,
        'block_2': block_2,
        'block_3': block_3,
        'hits': hits[hit_cols],
        'subqueries': subqueries,
        'inferred_ingredients': inferred_ingredients,
        'ingredient_catalog': ingredient_catalog,
        'ingredient_catalog_preview': _catalog_preview(ingredient_catalog, n=3),
        'cost_plan': cost_plan_df,
        'cost_summary': cost_summary,
        'cost_plan_text': cost_plan_text,
        'ingredient_catalog_text': ingredient_catalog_text,
        'raw_response': raw_resp_block3,
    }



In [9]:
question = 'Dame la receta de muslos de pollo con menestra de verduras'
result = ask_agent(
    question,
    top_k=35,
    retrieval_mode='hybrid',
    alpha=0.65,
    recipe_mode='auto',
    use_ingredient_tool=True,
    candidates_per_ingredient=12,
)
print(result['answer'])
result['cost_plan'].head(20)


Ingredientes de Mercadona:
- 1 ud Muslos de pollo deshuesados con piel
- 1 chorro Aceite de oliva 1º Hacendado
- 20 g Picada de ajo y perejil
- 150 g Salteado de pimientos y cebolla cortado y lavado
- 150 g Pimiento verde freír
- 200 g Tomate rallado
- 150 g Zanahorias
- 1 ud Judías verdes extrafinas Hacendado
- 1 ud Guisantes extra Hacendado
- 1 ud Calabacín verde
- 1 ud Patatas para microondas
- 1 cucharada Bicarbonato de sódio Hacendado

Precio de los productos a comprar para esta receta: 36,78€

Receta y pasos:
Calienta 0,060 l de aceite de oliva en una sartén grande y sofríe 0,020 kg de ajo y perejil picados junto con 0,150 kg de cebolla y pimiento verde hasta que estén tiernos. Añade 1 kg de muslos de pollo deshuesados con piel y dóralos por ambos lados. Incorpora 0,200 kg de tomate rallado y cocina a fuego medio. Mientras tanto, corta 0,150 kg de zanahorias, 1 kg de calabacín y 1 kg de patatas para microondas en trozos medianos. Añade también 1 kg de judías verdes y 1 kg de guis

,ingredient,required_qty,required_unit,requirement_source,product_id,product_name,unit_size,size_format,price_unit,units_to_buy,purchase_cost_eur,escandallo_cost_eur,notes
0,muslos de pollo,1.00,ud,fallback,2788.0,Muslos de pollo deshuesados con piel,0.50,kg,2.78,1,2.78,NaN,"Unidad no comparable, compra minima 1 unidad"
1,aceite de oliva,0.06,l,aceite de oliva,4641.0,Aceite de oliva 1º Hacendado,5.00,l,20.85,1,20.85,0.250200,OK unidades comparables
2,ajo,0.02,kg,ajo,40475.0,Picada de ajo y perejil,0.10,kg,1.50,1,1.50,0.300000,OK unidades comparables
3,cebolla,0.15,kg,cebolla,69423.0,Salteado de pimientos y cebolla cortado y lavado,0.40,kg,2.40,1,2.40,0.900000,OK unidades comparables
4,pimiento verde,0.15,kg,pimiento,69320.0,Pimiento verde freír,0.10,kg,0.25,2,0.50,0.375000,OK unidades comparables
5,tomate,0.20,kg,tomate,69938.0,Tomate rallado,0.29,kg,1.45,1,1.45,1.000000,OK unidades comparables
6,zanahoria,0.15,kg,zanahoria,69669.0,Zanahorias,0.50,kg,0.80,1,0.80,0.240000,OK unidades comparables
7,judias verdes,1.00,ud,fallback,20349.0,Judías verdes extrafinas Hacendado,0.53,kg,1.90,1,1.90,NaN,"Unidad no comparable, compra minima 1 unidad"
8,guisantes,1.00,ud,fallback,16416.0,Guisantes extra Hacendado,0.39,kg,0.95,1,0.95,NaN,"Unidad no comparable, compra minima 1 unidad"
9,calabacin,1.00,ud,fallback,69338.0,Calabacín verde,0.43,kg,1.05,1,1.05,NaN,"Unidad no comparable, compra minima 1 unidad"


## Notas
- Si cambias el Excel, vuelve a ejecutar la celda de `chunks` y luego `ensure_embeddings(..., force_rebuild=True)`.
- Puedes bajar coste usando menos `top_k` o cambiando a un modelo chat mas pequeno.
- Puedes guardar respuestas historicas en CSV si quieres trazabilidad de preguntas/respuestas.


In [8]:
import subprocess
import sys
import time
import urllib.request
import webbrowser
from pathlib import Path
from IPython.display import HTML, display

BASE_DIR = Path('/home/wencm/RecetONA')
FRONTEND_DIR = BASE_DIR / 'frontend'
RAG_SERVER_SCRIPT = BASE_DIR / 'local_rag_server.py'
RAG_HEALTH_URL = 'http://127.0.0.1:8787/health'
CHAT_URL = 'http://127.0.0.1:8080/main.html'


def _wait_http(url, timeout=30):
    end = time.time() + timeout
    while time.time() < end:
        try:
            with urllib.request.urlopen(url, timeout=2) as resp:
                if 200 <= resp.status < 500:
                    return True
        except Exception:
            pass
        time.sleep(0.5)
    return False


def _spawn_once(global_name, cmd, cwd):
    proc = globals().get(global_name)
    if proc is not None and proc.poll() is None:
        return proc, False

    proc = subprocess.Popen(cmd, cwd=str(cwd))
    globals()[global_name] = proc
    return proc, True


rag_proc, rag_started = _spawn_once(
    'recetona_rag_api_proc',
    [sys.executable, str(RAG_SERVER_SCRIPT), '--host', '127.0.0.1', '--port', '8787'],
    BASE_DIR,
)

if not _wait_http(RAG_HEALTH_URL, timeout=45):
    raise RuntimeError('No se pudo iniciar la API local RAG en http://127.0.0.1:8787')

front_proc, front_started = _spawn_once(
    'recetona_frontend_http_proc',
    [sys.executable, '-m', 'http.server', '8080', '--bind', '127.0.0.1'],
    FRONTEND_DIR,
)

if not _wait_http(CHAT_URL, timeout=20):
    raise RuntimeError('No se pudo iniciar el frontend local en http://127.0.0.1:8080/main.html')

if rag_started:
    print('API RAG local iniciada en: http://127.0.0.1:8787')
else:
    print('API RAG local ya estaba iniciada.')

if front_started:
    print('Frontend local iniciado en: http://127.0.0.1:8080/main.html')
else:
    print('Frontend local ya estaba iniciado.')

webbrowser.open(CHAT_URL)
display(HTML(f'<a href="{CHAT_URL}" target="_blank">Abrir chat RecetONA</a>'))
print('Chat abierto en el navegador.')


[2026-02-11 23:17:35,951] INFO Inicializando runtime desde notebook: /home/wencm/RecetONA/mercadona_rag_notebook.ipynb


Excel: /home/wencm/RecetONA/mercadona_data.xlsx
Cache: /home/wencm/RecetONA/rag_cache
Python: /home/wencm/WhaRAGBot/.venv/bin/python
OPENAI_API_KEY presente: True
Filas: 4553
Chunks guardados en: /home/wencm/RecetONA/rag_cache/chunks.csv
Embeddings cargados desde cache: /home/wencm/RecetONA/rag_cache/embeddings.npy


[2026-02-11 23:17:44,166] INFO API local escuchando en http://127.0.0.1:8787
[2026-02-11 23:17:44,166] INFO Endpoints: GET /health, POST /chat
[2026-02-11 23:17:44,450] INFO 127.0.0.1 - "GET /health HTTP/1.1" 200 -


Serving HTTP on 127.0.0.1 port 8080 (http://127.0.0.1:8080/) ...


127.0.0.1 - - [11/Feb/2026 23:17:44] "GET /main.html HTTP/1.1" 200 -


API RAG local iniciada en: http://127.0.0.1:8787
Frontend local iniciado en: http://127.0.0.1:8080/main.html


127.0.0.1 - - [11/Feb/2026 23:17:49] "GET /main.html HTTP/1.1" 200 -
[2026-02-11 23:17:49,936] INFO 127.0.0.1 - "GET /health HTTP/1.1" 200 -
127.0.0.1 - - [11/Feb/2026 23:17:50] code 404, message File not found
127.0.0.1 - - [11/Feb/2026 23:17:50] "GET /favicon.ico HTTP/1.1" 404 -


Chat abierto en el navegador.


[2026-02-11 23:18:05,920] INFO 127.0.0.1 - "OPTIONS /chat HTTP/1.1" 200 -
[2026-02-11 23:18:08,587] INFO HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
[2026-02-11 23:18:09,178] INFO HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
[2026-02-11 23:18:09,412] INFO HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
[2026-02-11 23:18:09,620] INFO HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
[2026-02-11 23:18:09,778] INFO HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
[2026-02-11 23:18:09,977] INFO HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
[2026-02-11 23:18:10,139] INFO HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
[2026-02-11 23:18:10,331] INFO HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
[2026-02-11 23:18:10,579] INFO HTTP Request: POST https://api.openai.com/v1/embeddings 